In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *
from matplotlib.lines import Line2D

import seaborn as sns

sns.set_theme()
sns.set_context("paper")
sns.set(rc={"xtick.bottom": True, "ytick.left": True})
# colors = sns.color_palette("mako").as_hex()
# my_cmap = sns.color_palette("viridis", as_cmap=True)

In [ ]:
params = np.load("./params.npy")
print(params)
# (pad_mode, pad_size)
residues_tmp = np.load(f"./residues/model_0.npy")
epochs = [10 * (i + 1) for i in range(len(residues_tmp))]
print(f"{len(params) = }   {epochs = }")

In [ ]:
pad_mode, pad_size = params.T
pad_mode = np.unique(pad_mode).astype(int)
pad_size = np.unique(pad_size).astype(int)
colors = list(sns.color_palette("deep").as_hex())
print(colors)

if not (os.path.exists("./images")):
    os.makedirs("./images")

In [ ]:
color_indices = [3, 0, 4]
line_type = ["-s", "--o", ":>"]
pad_mode_str = ["No padding", "Constant", "Reflect"]

legend_elements = [
    Line2D(
        [0],
        [0],
        color=colors[color_indices[i]],
        lw=4,
        label=f"{pad_mode_str[i]}",
    )
    for i in range(len(color_indices))
]

plt.figure(figsize=(6, 4))

for i in range(0, len(params)):
    pad_mode_i, pad_size_i = params[i]

    if pad_mode_i == 0.0 and pad_size_i == 0.0:
        count = 0
    elif pad_mode_i == 0 and pad_size_i != 0:
        count = 1
    else:
        count = 2

    if pad_size_i == 0.0:
        index = 0
    elif pad_size_i == 0.025:
        index = 0
    elif pad_size_i == 0.05:
        index = 1
    else:
        index = 2

    residues = np.load(f"./residues/model_{i}.npy")

    plt.plot(
        epochs,
        residues,
        line_type[index],
        color=colors[color_indices[count]],
    )

plt.yscale("log")
plt.legend(handles=legend_elements, ncol=1, fontsize=12, loc="upper right")
plt.xlabel("Epochs", fontsize=16)
plt.ylabel("Residues", fontsize=16)
plt.tight_layout()
plt.savefig("./images/change_padding.png")
plt.show()